In [ ]:
!pip install kaggle

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [18]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"sunidhisuri","key":"cfaf84f1b1d9d75c70382f7c4d273f67"}'}

In [19]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

 99% 1.20G/1.21G [00:10<00:00, 157MB/s]
100% 1.21G/1.21G [00:10<00:00, 126MB/s]


In [20]:
!unzip /content/gtzan-dataset-music-genre-classification.zip

Archive:  /content/gtzan-dataset-music-genre-classification.zip
  inflating: Data/features_30_sec.csv  
  inflating: Data/features_3_sec.csv  
  inflating: Data/genres_original/blues/blues.00000.wav  
  inflating: Data/genres_original/blues/blues.00001.wav  
  inflating: Data/genres_original/blues/blues.00002.wav  
  inflating: Data/genres_original/blues/blues.00003.wav  
  inflating: Data/genres_original/blues/blues.00004.wav  
  inflating: Data/genres_original/blues/blues.00005.wav  
  inflating: Data/genres_original/blues/blues.00006.wav  
  inflating: Data/genres_original/blues/blues.00007.wav  
  inflating: Data/genres_original/blues/blues.00008.wav  
  inflating: Data/genres_original/blues/blues.00009.wav  
  inflating: Data/genres_original/blues/blues.00010.wav  
  inflating: Data/genres_original/blues/blues.00011.wav  
  inflating: Data/genres_original/blues/blues.00012.wav  
  inflating: Data/genres_original/blues/blues.00013.wav  
  inflating: Data/genres_original/blues/blues

In [23]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=36301a7b24d463ca34084c5fedb2752b1e749e4d842d48e473d36d3d636ded06
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
#upload the Test.zip file to google colab

In [24]:
!unzip /content/Test.zip

Archive:  /content/Test.zip
replace Test/Test1.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/Test1.wav          
replace Test/Test2.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/Test2.wav          
replace Test/Test3.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/Test3.wav          
replace Test/Test4.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/Test4.wav          
replace Test/Test5.mp3? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test/Test5.mp3          


In [25]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile

import os
import pickle
import random
import operator

import math

In [63]:
# function to get the distance between feature vecotrs and find neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [26]:
# identify the class of the instance
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)

    return sorter[0][0]

In [27]:
# function to evaluate the model


def getAccuracy(testSet, prediction):
  correct = 0
  for x in range(len(testSet)):
    if testSet[x][-1] == predictions[x]:
      correct += 1
       
  return (1.0 * correct) / len(testSet)

In [28]:
#import os    
#directory = os.path.join('E:', os.sep, 'archive', 'Data', 'genres_original')
# directory that holds the wav files
directory = "/content/Data/genres_original"

# binary file where we will collect all the features extracted using mfcc (Mel Frequency Cepstral Coefficients)
f = open("my.dat", 'wb')

i = 0


for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(os.path.join(directory,folder)):        
        try:
            (rate, sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()


Got an exception:  File format b'\xcb\x15\x1e\x16'... not understood.  in folder:  jazz  filename:  jazz.00054.wav


In [29]:
# Split the dataset into training and testing sets respectively
dataset = []

def loadDataset(filename, split, trSet, teSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

In [30]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [32]:
print(len(testSet))

345


In [64]:
# making predictions using KNN
#from __future__ import division
leng = len(testSet)



#print(len(testSet))
predictions = []

for x in range(leng):
  
  predictions.append(nearestClass( getNeighbors(trainingSet, testSet[x], 5)))


accuracy1 =getAccuracy(testSet, predictions)
print(accuracy1)


0.7072463768115942


In [67]:
# testing the code with external samples
# URL: https://uweb.engr.arizona.edu/~429rns/audiofiles/audiofiles.html

test_dir = "/content/Test/"

#for file in os.listdir(os.path.join(directory,folder)):
test_file = test_dir + "Test5.wav"
#test_file = test_dir + "Test2.wav"
#test_file = test_dir + "Test4.wav"

In [39]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)
  


In [40]:
from collections import defaultdict
results = defaultdict(int)

directory = "./Data/genres_original"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [68]:
pred = nearestClass(getNeighbors(dataset, feature, 5))
print(results[pred])

jazz
